Here, you develop the exact spatial derivative operator and its unit test. (Develop a quantitative assessment between what you've already found here for the Fourier derivative and the known derivative.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh = 32
refRatio = 2
c = 1.
CFL = 0.5
nt = 1
RK = 4
deriv = 'CD'
order = 0
mu = 0.5
sigma = .1
wavenumber = 13


# Grid parameters.
nh_min = 64
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
CFL = np.sqrt(0.5)
nt = 50
deriv = 'CD'
RK = 4
order = 2

# Waveform parameters.
x_0 = 0.
x_1 = float(cfInter) / float(nh_min)
print(x_1)

# Physical parameters.
L = 1e-3
locs = []#x_1]
epsilons_r = [1]#, 11.68]
mus_r = [1]#, 0.99837]

# Pseudo-FFT parameters.
Hans = False

In [3]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)

physics = BT.PhysProperties(omega, epsilons_r, mus_r, locs, L)

degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
t = 0

In [4]:
# For Gaussian:
# waveform = WFT.Gauss(omega, sigma, mu)
# waveDeriv = WFT.Gauss(omega, sigma, mu, BooleAve = True, deriv = True)

# For Gaussian wavepacket:
waveform = WFT.WavePacket(omega, sigma, mu, wavenumber)
waveDeriv = WFT.WavePacket(omega, sigma, mu, wavenumber, deriv = True)

derivMat = OT.ExactSpatDerivOp(omega) # Works on fine grid to produce results on AMR grid!
spatOp = -cMat @ derivMat
# testDeriv = TT.ExactSpatDeriv(omega, t, waveform, -1, order)
# THIS IS A PROBLEM FOR NEW RK SETUP!!!

FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveform)
FCoefsDeriv = FFTT.FourierCoefs(waves @ nullspace, waveDeriv)
FCoefsTest = spatOp @ FCoefs# FFTT.FourierCoefs(waves @ nullspace, testDeriv)
allCoefs = PT.Load(FCoefs, FCoefsDeriv, FCoefsTest)
labels = ['Initial Waveform', 'Exact Derivative', 'Fourier Derivative']

AttributeError: module 'Modules.OperatorTools' has no attribute 'ExactSpatDeriv'

In [ ]:
PT.PlotMixedWave(omega, waves, nullspace @ allCoefs, rescale = 6, yGrid = True, labels = labels)#, saveName = 'GaussTest', dpi = 300)
